In [67]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from math import log, sqrt
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn import svm
import joblib
import datetime


data_raw = pd.read_csv("./taxi_trip_pricing.csv")

In [68]:
#Fill data from formula of Price = Minute_rate*Minutes + Base + dist_rate*dist
data_fare_fix = data_raw.copy()
data_fare_fix   ['Trip_Price'] = data_fare_fix ['Trip_Price'].fillna(data_fare_fix  ['Base_Fare'] + (
    data_fare_fix   ['Per_Km_Rate']*data_fare_fix  ['Trip_Distance_km']) + (data_fare_fix    ['Per_Minute_Rate']*data_fare_fix   ['Trip_Duration_Minutes']))
data_fare_fix   ['Base_Fare'] = data_fare_fix  ['Base_Fare'].fillna(data_fare_fix    ['Trip_Price'] - (
    data_fare_fix   ['Per_Km_Rate']*data_fare_fix  ['Trip_Distance_km']) + (data_fare_fix    ['Per_Minute_Rate']*data_fare_fix   ['Trip_Duration_Minutes']))
data_fare_fix   ['Per_Km_Rate'] = data_fare_fix    ['Per_Km_Rate'].fillna(
    (data_fare_fix  ['Trip_Price'] - (data_fare_fix   ['Per_Minute_Rate']*data_fare_fix  ['Trip_Duration_Minutes']) - data_fare_fix    ['Base_Fare'])/data_fare_fix    ['Trip_Distance_km'])
data_fare_fix   ['Trip_Distance_km'] = data_fare_fix   ['Trip_Distance_km'].fillna(
    (data_fare_fix  ['Trip_Price'] - (data_fare_fix   ['Per_Minute_Rate']*data_fare_fix  ['Trip_Duration_Minutes']) - data_fare_fix    ['Base_Fare'])/data_fare_fix    ['Per_Km_Rate'])
data_fare_fix   ['Per_Minute_Rate'] = data_fare_fix    ['Per_Minute_Rate'].fillna(
    (data_fare_fix  ['Trip_Price'] - (data_fare_fix   ['Per_Km_Rate']*data_fare_fix  ['Trip_Distance_km']) - data_fare_fix ['Base_Fare'])/data_fare_fix ['Trip_Duration_Minutes'])
data_fare_fix   ['Trip_Duration_Minutes'] = data_fare_fix  ['Trip_Duration_Minutes'].fillna(
    (data_fare_fix  ['Trip_Price'] - (data_fare_fix   ['Per_Km_Rate']*data_fare_fix  ['Trip_Distance_km']) - data_fare_fix ['Base_Fare'])/data_fare_fix ['Per_Minute_Rate'])

In [69]:
#drop rows which are still Na in the above columns
data_fare_fix = data_fare_fix.dropna(subset=['Trip_Price', 'Base_Fare', 'Per_Km_Rate', 'Trip_Distance_km', 'Per_Minute_Rate', 'Trip_Duration_Minutes'])

In [70]:
#convert time of the day and traffic to numerical
data_seriel_cat_fix = data_fare_fix.copy()
data_seriel_cat_fix['Time_of_Day'] = data_fare_fix['Time_of_Day'].replace(["Morning", "Afternoon", "Evening", "Night"], [1,2,3, 4])
data_seriel_cat_fix['Traffic_Conditions'] = data_seriel_cat_fix["Traffic_Conditions"].replace(['Low', 'Medium', 'High'], [1,2,3])

C:\Users\Mayur\AppData\Local\Temp\ipykernel_12468\4069568324.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_seriel_cat_fix['Time_of_Day'] = data_fare_fix['Time_of_Day'].replace(["Morning", "Afternoon", "Evening", "Night"], [1,2,3, 4])
C:\Users\Mayur\AppData\Local\Temp\ipykernel_12468\4069568324.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_seriel_cat_fix['Traffic_Conditions'] = data_seriel_cat_fix["Traffic_Conditions"].replace(['Low', 'Medium', 'High'], [1,2,3])


In [71]:
#One Hot encode non Seriel Columns
data_hot_encoded = pd.get_dummies(data_seriel_cat_fix, columns=['Day_of_Week', 'Weather'], dtype=float, drop_first=True)


In [72]:
#Drop Any Remaining Na Values
data_clean = data_hot_encoded.dropna()

In [73]:
#Drop Base Fare, and rates as the model can cheat with it
data_clean = data_clean.drop(['Base_Fare', 'Per_Km_Rate', 'Per_Minute_Rate'], axis = 1)

In [74]:
#Deal With Tail Heavy Distributions
data_clean = data_clean.drop(data_clean[data_clean['Trip_Distance_km'] > 56].index)

In [75]:
#Split Train And Test before scaling
X = data_clean.drop(['Trip_Price'], axis=1).values
y = data_clean['Trip_Price'].values
scaler = StandardScaler()
X_train , X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42) 
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [76]:
#Use Linear ridge regression
reg = linear_model.Ridge()
param_grid = {"alpha": np.arange(0, 10, 0.1)}

grid_search = GridSearchCV(reg, param_grid, cv=10, scoring='neg_root_mean_squared_error')
grid_search.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=Ridge(),
             param_grid={'alpha': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
       2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8,
       3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9, 5. , 5.1,
       5.2, 5.3, 5.4, 5.5, 5.6, 5.7, 5.8, 5.9, 6. , 6.1, 6.2, 6.3, 6.4,
       6.5, 6.6, 6.7, 6.8, 6.9, 7. , 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7,
       7.8, 7.9, 8. , 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9. ,
       9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9])},
             scoring='neg_root_mean_squared_error')

In [77]:
#Check Usability of grid Search
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
62,0.000044,0.000132,0.001498,0.004299,6.2,{'alpha': 6.2},-14.699081,-16.174845,-13.357964,-16.068658,-13.658204,-14.672443,-13.798114,-13.583913,-16.147867,-12.122109,-14.428320,1.303777,1
61,0.000000,0.000000,0.000000,0.000000,6.1,{'alpha': 6.1000000000000005},-14.698828,-16.175073,-13.359018,-16.068334,-13.657695,-14.672431,-13.798131,-13.583921,-16.147902,-12.121866,-14.428320,1.303751,2
63,0.000000,0.000000,0.000000,0.000000,6.3,{'alpha': 6.300000000000001},-14.699334,-16.174617,-13.356911,-16.068983,-13.658714,-14.672456,-13.798098,-13.583906,-16.147832,-12.122352,-14.428320,1.303803,3
60,0.000674,0.001654,0.000237,0.000375,6.0,{'alpha': 6.0},-14.698577,-16.175303,-13.360074,-16.068011,-13.657185,-14.672420,-13.798148,-13.583929,-16.147938,-12.121624,-14.428321,1.303725,4
64,0.001656,0.004632,0.000152,0.000322,6.4,{'alpha': 6.4},-14.699588,-16.174389,-13.355860,-16.069308,-13.659225,-14.672470,-13.798083,-13.583900,-16.147798,-12.122596,-14.428322,1.303829,5


In [78]:
#Get final RMSE of test of Ridge Regressor
reg_opt = grid_search.best_estimator_
reg_test_pred = reg_opt.predict(X_test)
reg_opt_rmse = mean_squared_error(y_test, reg_test_pred)
sqrt(reg_opt_rmse)

16.119094278022832

In [79]:
#Setup and Eval SVM 
svm_est = svm.SVR(kernel="rbf")
svm_est.fit(X_train, y_train)
svm_pred = svm_est.predict(X_test)
svm_rmse = mean_squared_error(y_test, svm_pred)
sqrt(svm_rmse)

18.814372804092216

In [80]:
#Save models
joblib.dump(svm_est, "./models/SVM.pkl")
joblib.dump(reg_opt, "./models/Ridge.pkl")

['./models/Ridge.pkl']

**RIDGE REGRESSOR IS BETTER**

# START NEURAL NETWORKS

In [81]:
y_train.shape

(646,)

In [82]:
#imports
import tensorflow as tf
from tensorflow import keras
from keras import layers
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [83]:
#function to help batch the data
def pandas_to_dataset(dataframe, labels, batch_size=32, ):
    dataframe = dataframe.copy()
    # To transform the DataFrame into a key-value pair.
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))


    ds = ds.batch(batch_size)
    return ds

In [84]:
#Prep data for neural network
""" batch_size = 38
train_ds = pandas_to_dataset(X_train, y_train, batch_size=batch_size)
test_ds = pandas_to_dataset(X_test, y_test, batch_size=27) """

' batch_size = 38\ntrain_ds = pandas_to_dataset(X_train, y_train, batch_size=batch_size)\ntest_ds = pandas_to_dataset(X_test, y_test, batch_size=27) '

In [85]:
#Compile and Fit Model
neural_network = tf.keras.Sequential([layers.Dense(8, activation='relu'),
                                    layers.Dense(8, activation='relu'),
                                    layers.Dense(4, activation='relu'),
                                    layers.Dense(2, activation='relu'),
                                    layers.Dropout(0.1),
                                    layers.Dense(1)])
neural_network.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
log_dir = "./logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [86]:
neural_network.fit(np.array(X_train), np.array(y_train), batch_size=38, validation_data=(X_test, y_test), epochs=150, callbacks=[tensorboard_callback])
%tensorboard --logdir ./logs/fit

Epoch 1/150
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.0000e+00 - loss: 3317.8372 - val_accuracy: 0.0000e+00 - val_loss: 3537.1262
Epoch 2/150
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 3298.2419 - val_accuracy: 0.0000e+00 - val_loss: 3531.5491
Epoch 3/150
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 3267.1023 - val_accuracy: 0.0000e+00 - val_loss: 3528.2805
Epoch 4/150
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 3327.2288 - val_accuracy: 0.0000e+00 - val_loss: 3525.7183
Epoch 5/150
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 3216.7573 - val_accuracy: 0.0000e+00 - val_loss: 3523.3535
Epoch 6/150
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.0000e+00 - loss: 3383.4724 - val_accuracy: 0.0000e+00 - val_loss: 3521.3489
Epoch 7/150
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.0000e+00 - loss: 3429.4607 - val_accuracy: 0.0000e+00 - val_loss: 3519.3843
Epoch 8/150


Reusing TensorBoard on port 6008 (pid 11096), started 0:37:23 ago. (Use '!kill 11096' to kill it.)

In [87]:
neural_network_pred = neural_network.predict(X_test)
neural_network_rmse = mean_squared_error(y_test, neural_network_pred)
neural_network_pred

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


array([[2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],
       [2.5198748],


# AS THE AMOUNT OF DATA IS LESS NEURAL NETWORKS DO NOT PERFORM AS WELL AS CONVENTIONAL MACHINE LEARNING ALGORITHIMS